In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

mufex_main = Mufex(use_test_net=False)

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(exchange="binance_us", symbol="BTCUSDT", timeframe="1h", candles_to_dl=200)
open = candles[:, 1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4], 1)
candles_to_df(candles)

,timestamp,open,high,low,close
datetime,,,,,
2023-11-05 10:00:00,1699178400000,35066.16,35115.20,35019.59,35103.40
2023-11-05 11:00:00,1699182000000,35109.00,35129.26,35023.95,35042.54
2023-11-05 12:00:00,1699185600000,35046.27,35146.60,35019.63,35078.77
2023-11-05 13:00:00,1699189200000,35078.73,35109.46,34790.75,34840.80
2023-11-05 14:00:00,1699192800000,34840.80,34927.67,34794.74,34903.47
...,...,...,...,...,...
2023-11-13 13:00:00,1699880400000,36943.71,36951.96,36791.51,36818.42
2023-11-13 14:00:00,1699884000000,36847.37,36921.01,36511.00,36798.28
2023-11-13 15:00:00,1699887600000,36801.03,36885.00,36642.15,36840.53


In [3]:
super_trend = supertrend_tv(candles=candles)

In [4]:
plot_supertrend(candles=candles, super_trend=super_trend)